# Analysis of a Twitter Social Network

In this section we are going to parse the tweets we collected and build the social network of interactions between Twitter users. We will also see how to analyze the network using NetworkX. We will look at the different component of the network and at percolation processes on this network.

## Parsing tweets

Tweets are saved in JSON format ([JavaScript Object Notation](https://www.w3schools.com/js/js_json_intro.asp))
JSON is text, written with JavaScript object notation.

The `json` python module allows to easily import json file into python [Dictonairies](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)



In [ ]:
#load tweets 

import json


filename = 'tweets_covid.txt'

tweet_list = []

with open(filename, 'r') as fopen:
    # each line correspond to a tweet
    for line in fopen:
        tweet_list.append(json.loads(line))
        

Let's look at the informations contained in a tweet

In [ ]:
# take the first tweet of the list
tweet = tweet_list[0]

In [ ]:
# each tweet is a python dictionary
type(tweet)

In [ ]:
# all the 'entries' of the dictionary
tweet.keys()

you can find a description of the fields in the Twitter API documentation: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [ ]:
#creation time
tweet['created_at']

In [ ]:
# text of the tweet
print(tweet['text'])

In [ ]:
# user info
tweet['user']

In [ ]:
# user is itslef a dict
print(type(tweet['user']))

tweet['user']['name']

In [ ]:
# unique id of the user
tweet['user']['id']

In [ ]:
#is the tweet a retweet?
'retweeted_status' in tweet

In [ ]:
if 'retweeted_status' in tweet:
    print(tweet['retweeted_status'])
# the `retweeted_status` is also a tweet dictionary    

In [ ]:
if 'retweeted_status' in tweet:
    print(tweet['retweeted_status']['text'])

In [ ]:
# user id and name of the retweeted user?
if 'retweeted_status' in tweet:
    print(tweet['retweeted_status']['user']['id'])
    print(tweet['retweeted_status']['user']['name'])

In [ ]:
# is the tweet a reply?
'in_reply_to_user_id' in tweet and tweet['in_reply_to_user_id'] is not None

In [ ]:
# 'entities' contains the hashtags, urls and usernames used in the tweet
tweet['entities']

In [ ]:
# user id of the mentioned users
for  mention in tweet['entities']['user_mentions']:
    print(mention['id'])

In [ ]:
# is the tweet a quote?
'quoted_status' in tweet

# Building the network of interactions

We will use the python module [`NetworkX`](https://networkx.readthedocs.io/en/stable/index.html) to construct and analyze the social network.

A short introduction to networkx: https://networkx.org/documentation/stable/reference/introduction.html


There are four types of interactions between two users in Twitter:
- Retweet
- Quote
- Reply
- Mention

In [ ]:
# let's define some functions to extract the interactions from tweets

def getTweetID(tweet):
    """ If properly included, get the ID of the tweet """
    return tweet.get('id')
    
def getUserIDandScreenName(tweet):
    """ If properly included, get the tweet 
        user ID and Screen Name """
    user = tweet.get('user')
    if user is not None:
        uid = user.get('id')
        screen_name = user.get('screen_name')
        return uid, screen_name
    else:
        return (None, None)

def getRetweetedUserIDandSreenName(tweet):
    """ If properly included, get the retweet 
        source user ID and Screen Name"""
    
    retweet = tweet.get('retweeted_status')
    if retweet is not None:
        return getUserIDandScreenName(retweet)
    else:
        return (None, None)
    
def getRepliedUserIDandScreenName(tweet):
    """ If properly included, get the ID and Screen Name 
        of the user the tweet replies to """
    
    reply_id = tweet.get('in_reply_to_user_id')
    reply_screenname = tweet.get('in_reply_to_screen_name')
    return reply_id, reply_screenname
    
def getUserMentionsIDandScreenName(tweet):
    """ If properly included, return a list of IDs and Screen Names tuple
        of all user mentions, including retweeted and replied users """
        
    mentions = []
    entities = tweet.get('entities')
    if entities is not None:
        user_mentions = entities.get('user_mentions')
        for mention in user_mentions:
            mention_id = mention.get('id')
            screen_name = mention.get('screen_name')
            mentions.append((mention_id, screen_name))
    
    return mentions

    
def getQuotedUserIDandScreenName(tweet):
    """ If properly included, get the ID of the user the tweet is quoting"""
    
    quoted_status = tweet.get('quoted_status')
    
    if quoted_status is not None:
        return getUserIDandScreenName(quoted_status)
    else:
        return (None, None)
    
def getAllInteractions(tweet):
    """ Get all the interactions from this tweet
    
        returns : (tweeter_id, tweeter_screenname), list of (interacting_id, interacting_screenname)
    """
    
    # Get the tweeter
    tweeter = getUserIDandScreenName(tweet)
    
    # Nothing to do if we couldn't get the tweeter
    if tweeter[0] is None:
        return (None, None), []
    
    # a python set is a collection of unique items
    # we use a set to avoid duplicated ids
    interacting_users = set()
    
    # Add person they're replying to
    interacting_users.add(getRepliedUserIDandScreenName(tweet))
    
    # Add person they retweeted
    interacting_users.add(getRetweetedUserIDandSreenName(tweet))
    
    # Add person they quoted
    interacting_users.add(getQuotedUserIDandScreenName(tweet))
    
    # Add mentions
    interacting_users.update(getUserMentionsIDandScreenName(tweet))
  
    # remove the tweeter if he is in the set
    interacting_users.discard(tweeter)
    # remove the None case
    interacting_users.discard((None,None))
    
    # Return our tweeter and their influencers
    return tweeter, list(interacting_users)
    


In [ ]:
print(getUserIDandScreenName(tweet_list[3]))
print(getAllInteractions(tweet_list[4]))

tweet_list[100].get('text')

#### Let's build the network

In [ ]:
import networkx as nx

# define an empty Directed Graph
# A directed graph is a graph where edges have a direction
# in our case the edges goes from user that sent the tweet to
# the user with whom they interacted (retweeted, mentioned or quoted)
G = nx.DiGraph()

# loop over all the tweets and add edges if the tweet include some interactions
for tweet in tweet_list:
    # find all influencers in the tweet
    tweeter, interactions = getAllInteractions(tweet)
    tweeter_id, tweeter_name = tweeter
    tweet_id = getTweetID(tweet)
    
    # add an edge to the Graph for each influencer
    for interaction in interactions:
        interact_id, interact_name = interaction
        
        # add edges between the two user ids
        # this will create new nodes if the nodes are not already in the network
        # we also add an attribute the to edge equal to the id of the tweet
        G.add_edge(tweeter_id, interact_id, tweet_id=tweet_id)
        
        # add name as a property to each node
        # with networkX each node is a dictionary
        G.nodes[tweeter_id]['name'] = tweeter_name
        G.nodes[interact_id]['name'] = interact_name
        

In [ ]:
# The graph's node are contained in a NodeView which has a dict-like interface
print(type(G.nodes))

In [ ]:
# the keys are the user_id
nodelist = list(G.nodes.keys())
print(nodelist)

In [ ]:
# each node is itself a dictionary with node attributes as key,value pairs
print(type(G.nodes[nodelist[0]]))
print(G.nodes[nodelist[0]])

In [ ]:
# edges are contained in a EdgeView with a set-like interface
print(type(G.edges))
print(G.edges())

In [ ]:
# we can see all the edges going out of this node
# each edge is a dictionary inside this dictionary with a key 
# corresponding to the target user_id
e = G.out_edges(nodelist[11], data=True)
print(nodelist[11])
print(e)

In [ ]:
# we can iterate over the out-edges 
for s,t,data in e:
    print(s,t,data)

#### Some basic properties of the Network:

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
# listing all nodes 
nodelist = list(G.nodes())

nodelist[:3]

In [ ]:
# degree of a node
print(G.degree(nodelist[2]))
print(G.in_degree(nodelist[2]))
print(G.out_degree(nodelist[2]))

In [ ]:
# dictionary with the degree of all nodes
all_degrees = [G.degree(n) for n in nodelist] # this is the degree for undirected edges
in_degrees = [G.in_degree(n) for n in nodelist]
out_degrees = [G.out_degree(n) for n in nodelist]

In [ ]:
# average degree
2*G.number_of_edges()/G.number_of_nodes()

In [ ]:
import numpy as np
np.array(all_degrees).mean()

In [ ]:
np.array(in_degrees).mean()

In [ ]:
np.array(out_degrees).mean()

In [ ]:
# maximum degree
max(all_degrees)

In [ ]:
# we want to make a list with (user_id, username, degree) for all nodes
degree_node_list = []
for node in nodelist:
    degree_node_list.append((node, G.nodes[node]['name'], G.degree(node)))
    
print('Unordered user, degree list')    
print(degree_node_list[:10])

# sort the list according the degree in descinding order
degree_node_list = sorted(degree_node_list, key=lambda x:x[2], reverse=True)
print('Ordered user, degree list')    
print(degree_node_list[:10])

In [ ]:
# we need to import matplolib for making plots
# and numpy for numerical computations
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Network components

For **directed** graphs we can define two types of components:
- Weakly connected components
- Strongly connected components

Weakly connected component (WCC): maximal set of nodes where there exists a path in at least one direction between each pair of nodes.

Strongly connected component (SCC): maximal set of nodes where there exists a path in both directions between each pair of nodes.

Weakly connected giant (largest) component (WCGC): Largest WCC
Strongly connected giant (largest) component (SCGC): Largest SCC

<img src="network_components.svg" style="width: 250px;"/>

In [ ]:
# this returns a list of set of nodes belonging to the 
# different (weakly) connected components
components = list(nx.weakly_connected_components(G))

# sort the component according to their size
components = list(sorted(components, key=lambda x:len(x), reverse=True))

In [ ]:
# make a list with the size of each component
comp_sizes = []
for comp in components:
    comp_sizes.append(len(comp))

In [ ]:
# plot the histogram of component sizes
hist = plt.hist(comp_sizes, bins=100)

In [ ]:
# histogram with logarithmic y scale
hist = plt.hist(comp_sizes, bins=100, log=True)
tx = plt.xlabel('component size')
ty = plt.ylabel('number of components')

In [ ]:
# sizes of the ten largest components
comp_sizes[:10]

In [ ]:
# let's make a new graph which is the subgraph of G corresponding to 
# the largest connected component
# let's find the largest component
largest_comp = components[0]
LCC = G.subgraph(largest_comp)

In [ ]:
G.number_of_nodes()

In [ ]:
LCC.number_of_nodes()

In [ ]:
# let's plot the degree distribution inside the LCC
degrees = [LCC.degree(n) for n in LCC.nodes()]
degrees

In [ ]:
degree_array = np.array(degrees)
hist = plt.hist(degree_array, bins=100)

In [ ]:
# using logarithmic scales
hist = plt.hist(degree_array, bins=100, log=True)
plt.xscale('log')


In [ ]:
# logarithmic scale with logarithmic bins
N, bins, patches = plt.hist(degree_array, bins=np.logspace(0,np.log10(degree_array.max()+1), 20), log=True)
plt.xscale('log')
tx = plt.xlabel('k - degree')
ty= plt.ylabel('number of nodes')


In [ ]:
# Degree probability distribution (P(k))

# since we have logarithmic bins, we need to
# take into account the fact that the bins 
# have different lenghts when normalizing
bin_lengths = np.diff(bins) # lenght of each bin

summ = np.sum(N*bin_lengths)
normalized_degree_dist = N/summ

# check normalization:
print(np.sum(normalized_degree_dist*bin_lengths))

hist = plt.bar(bins[:-1], normalized_degree_dist, width=np.diff(bins))
plt.xscale('log')
plt.yscale('log')
tx = plt.xlabel('k (degree)')
ty = plt.ylabel('P(k)')

### Exercise: do the same for the Graph comprising only retweet, replies, quote and mentions

### Percolation of the Giant Component

In [ ]:
import random

def getGCsize(G):
    """ returns the size of the largest component of G"""
        
    return len(max(nx.connected_components(G), key=len))
    


#### Random Attack:

In [ ]:
# list that will contain the size of the GC as we remove nodes
rnd_attack_GC_sizes = []

# we take into account the undirected version of the graph
LCCundirected = nx.Graph(LCC)

nodes_list = list(LCCundirected.nodes())


while len(nodes_list) > 1:
    # add the size of the  current GC
    rnd_attack_GC_sizes.append(getGCsize(LCCundirected))
    
    # pick a random node
    rnd_node = random.choice(nodes_list)
    # remove from graph
    LCCundirected.remove_node(rnd_node)
    # remove from node list
    nodes_list.remove(rnd_node)


In [ ]:
# convert list to numpy array
rnd_attack_GC_sizes = np.array(rnd_attack_GC_sizes)

# normalize by the initial size of the GC
GC_rnd = rnd_attack_GC_sizes/rnd_attack_GC_sizes[0]

# fraction of removed nodes
q = np.linspace(0,1,num=GC_rnd.size)

plt.plot(q,GC_rnd)
tx = plt.xlabel('q')
ty = plt.ylabel('GC')


#### High degree attack:

In [ ]:
# high degree attack
LCCundirected = nx.Graph(LCC)

# list of pairs (node, degree) sorted according the degree
node_deg_dict = dict(nx.degree(LCCundirected))
nodes_sorted = sorted(node_deg_dict, key=node_deg_dict.get)

# list that will contain the size of the GC as we remove nodes
hd_attack_GC_sizes = []

while len(nodes_sorted) > 1:
    
    hd_attack_GC_sizes.append(getGCsize(LCCundirected))
    
    #remove node according to their degree
    node = nodes_sorted.pop() # pop() removes and returns the last element
    LCCundirected.remove_node(node)
    
    



In [ ]:
hd_attack_GC_sizes = np.array(hd_attack_GC_sizes)
GC_hd = hd_attack_GC_sizes/hd_attack_GC_sizes[0]
q = np.linspace(0,1,num=GC_hd.size)

plt.plot(q,GC_rnd, label='random attack')
plt.plot(q,GC_hd, label='High-Degree attack')
tx = plt.xlabel('q')
ty = plt.ylabel('GC')
_ = plt.legend()


#### Exercise: implement the High-Degree Adaptative (HDA) attack where at each step the node with the highest degree of the remaining graph is removed.

### PageRank

The *PageRank* centrality modifies the classical random walk by introducing a "teleportation" probability, i.e. at each step, the walkers have a given probability to teleport uniformly at random to any other nodes of the network.
This makes the random walk ergodic, i.e. it converges to a stationary distribution, even in directed and disconnected networks.

The update equation for the PageRank probability density is given by 

$\mathbf{p}(n+1) = (1-\alpha)\mathbf{p}(n)\mathbf{D}_\text{out}^{-1}\mathbf{A} + \frac{\alpha}{N}\mathbf{1} = \mathbf{p}(n) \mathbf{M}$

with

$ \mathbf{M} = (1-\alpha)\mathbf{D}_\text{out}^{-1}\mathbf{A} + \frac{\alpha}{N}\mathbf{1}^T\mathbf{1}$

In [ ]:
#teleportation probability
alpha = 0.15

#adjacency matrix
nodelist = list(G.nodes())
A = nx.to_numpy_array(G, nodelist=nodelist)

#diagonal matrix of out degrees
deg_out_vect = np.array([float(max(G.out_degree(n),1)) for n in nodelist])
D_out_inv = np.diag(1/deg_out_vect)

# teleportation transition matrix
N = A.shape[1]
S = np.ones((N,N))*1/N

# full transition matrix
M = (1-alpha)*D_out_inv @ A + alpha*S

# for dangling nodes (nodes without out-edges), we force the random teleportation
dangling_nodes = np.where(A.sum(1) == 0)[0]
M[dangling_nodes,:] = S[dangling_nodes,:]

#initial walker distribution and 1st iteration
p_last = np.ones(N)*1/N
p = np.matmul(p_last, M)

# iterate until sufficient convergence
eps = 1.0e-8
i = 1
while np.linalg.norm(p - p_last, 2) > eps:
        p_last = p
        p = np.matmul(p, M)
        i += 1

print(i)

In [ ]:
pg_ranking = np.array(np.argsort(p)[::-1])

pagerank_values = p[pg_ranking]
nodes_pagerank = [nodelist[r] for r in pg_ranking]
nodes_pagerank[:10]

In [ ]:
names_pagerank = [G.nodes[n]['name'] for n in nodes_pagerank]
names_pagerank[:10]

In [ ]:
hist = plt.bar(np.arange(p.shape[0]),np.sort(p)[::-1])
ty = plt.ylabel('PageRank value')
tx = plt.xlabel('PageRank ranking')

In [ ]:
# pagerank is a probability density
pagerank_values.sum()


In [ ]:
# draw the network of the top 100 nodes
nx.draw(G, nodelist=nodes_pagerank[:100], node_size=8000*pagerank_values[:100],width=0.5, arrows=False)

### Save the graph to a GEXF file:

GEFX is file format based on XML useful for exchanging files between softwares.

https://gephi.org/gexf/format/

In [ ]:
# First let's add the pagerank value as a node attribute
for n, pr in zip(nodes_pagerank,pagerank_values):
    if n in LCC:
        LCC.nodes[n]['page_rank'] = pr


In [ ]:
nx.write_gexf(LCC, 'twitter_lcc.gexf')

We can now open the file with [Gephi](https://gephi.org/) to vizualize the graph